# 1. 라이브러리 임포트

In [1]:
# 저장한 모델 불러오기
import pandas as pd
from numpy import loadtxt
import xgboost
import pickle
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#pickle.dump(final_model, open("pima.pickle.dat", "wb"))
kj_model = pickle.load(open("pima.pickle.dat", "rb"))

In [2]:
# 데이터 더미화 및 통합데이터 생성
def callDataone(data):
    del data["Loan ID"]
    del data["Customer ID"]
    del data["Months since last delinquent"]
    data = data.dropna()
    # 범주형 데이터 확인
    categorical_subset = data[['Term', 'Years in current job', 'Home Ownership', 'Purpose']]

    # one hot encoding(더미함수 사용)
    categorical_subset = pd.get_dummies(categorical_subset)

    data.drop(labels=['Term', 'Years in current job', 'Home Ownership', 'Purpose'], axis=1, inplace=True)
    data = pd.concat([data, categorical_subset], axis = 1)
    data = pd.concat([data,test],sort=False)
    #data = pd.concat([temp,data],sort=False)
    data = data.fillna(0)
    return data

In [3]:
# 고객 예상 데이터 로드
test = pd.read_csv("dataset/train_test.csv")
test

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,...,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,445412.0,709.0,1167493.0,5214.74,17.2,6.0,1.0,228190.0,416746.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,99999999.0,741.0,2231892.0,29200.53,14.9,18.0,1.0,297996.0,750090.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,347666.0,721.0,806949.0,8741.90,12.0,9.0,0.0,256329.0,386958.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,206602.0,7290.0,896857.0,16367.74,17.3,6.0,0.0,215308.0,272448.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,217646.0,730.0,1184194.0,10855.08,19.6,13.0,1.0,122170.0,272052.0,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77266,210584.0,719.0,783389.0,3727.61,17.4,6.0,0.0,456.0,259160.0,0.0,...,0,1,0,0,0,0,0,0,0,0
77267,147070.0,725.0,475437.0,2202.86,22.3,5.0,0.0,47766.0,658548.0,0.0,...,0,0,0,0,0,1,0,0,0,0
77268,99999999.0,732.0,1289416.0,13109.05,9.4,22.0,0.0,153045.0,509234.0,0.0,...,0,0,0,0,0,0,0,0,0,0
77269,103136.0,742.0,1150545.0,7315.57,18.8,12.0,1.0,109554.0,537548.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77271 entries, 0 to 77270
Data columns (total 44 columns):
Current Loan Amount               77271 non-null float64
Credit Score                      77271 non-null float64
Annual Income                     77271 non-null float64
Monthly Debt                      77271 non-null float64
Years of Credit History           77271 non-null float64
Number of Open Accounts           77271 non-null float64
Number of Credit Problems         77271 non-null float64
Current Credit Balance            77271 non-null float64
Maximum Open Credit               77271 non-null float64
Bankruptcies                      77271 non-null float64
Tax Liens                         77271 non-null float64
Term_Long Term                    77271 non-null int64
Term_Short Term                   77271 non-null int64
Years in current job_1 year       77271 non-null int64
Years in current job_10+ years    77271 non-null int64
Years in current job_2 years      77271 non-

In [4]:
test_new = pd.read_csv('dataset/train_new.csv')
test_new

,Loan ID,Customer ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,f8fa11bd-d0d2-4c9f-998e-9f9d866803a7,ab86ebb2-c4f4-47dd-b430-3d6270394f8b,468908,Long Term,6470,1833842,8 years,Rent,other,5898.93,14.4,NaN,19,1,183863,809490,1,0


In [5]:
test_new_one = callDataone(test_new)
test_new_one

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,...,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,468908.0,6470.0,1833842.0,5898.93,14.4,19.0,1.0,183863.0,809490.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,445412.0,709.0,1167493.0,5214.74,17.2,6.0,1.0,228190.0,416746.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,99999999.0,741.0,2231892.0,29200.53,14.9,18.0,1.0,297996.0,750090.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,347666.0,721.0,806949.0,8741.90,12.0,9.0,0.0,256329.0,386958.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,206602.0,7290.0,896857.0,16367.74,17.3,6.0,0.0,215308.0,272448.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77266,210584.0,719.0,783389.0,3727.61,17.4,6.0,0.0,456.0,259160.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77267,147070.0,725.0,475437.0,2202.86,22.3,5.0,0.0,47766.0,658548.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77268,99999999.0,732.0,1289416.0,13109.05,9.4,22.0,0.0,153045.0,509234.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77269,103136.0,742.0,1150545.0,7315.57,18.8,12.0,1.0,109554.0,537548.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
test_new_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77272 entries, 0 to 77270
Data columns (total 44 columns):
Current Loan Amount               77272 non-null float64
Credit Score                      77272 non-null float64
Annual Income                     77272 non-null float64
Monthly Debt                      77272 non-null float64
Years of Credit History           77272 non-null float64
Number of Open Accounts           77272 non-null float64
Number of Credit Problems         77272 non-null float64
Current Credit Balance            77272 non-null float64
Maximum Open Credit               77272 non-null float64
Bankruptcies                      77272 non-null float64
Tax Liens                         77272 non-null float64
Term_Long Term                    77272 non-null int64
Years in current job_8 years      77272 non-null int64
Home Ownership_Rent               77272 non-null int64
Purpose_other                     77272 non-null int64
Term_Short Term                   77272 non-

In [9]:
import pandas as pd
from flask import Flask
from flask_restful import Resource, Api, reqparse, request
from flask_cors import CORS
app = Flask(__name__)
api = Api(app)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
 
class RegistUser(Resource):
    def post(self):
        parser = reqparse.RequestParser()
        parser.add_argument('loan_id', type=str)
        parser.add_argument('customer_id', type=str)
        parser.add_argument('current_loan_amount', type=str)
        parser.add_argument('term', type=str)
        parser.add_argument('credit_score', type=str)
        parser.add_argument('annual_income', type=str)
        parser.add_argument('years_in_current_job', type=str)
        parser.add_argument('home_ownership', type=str)
        parser.add_argument('purpose', type=str)
        parser.add_argument('monthly_dept', type=str)
        parser.add_argument('years_of_credit_history', type=str)
        parser.add_argument('months_since_last_delinquent', type=str)
        parser.add_argument('number_of_open_accounts', type=str)
        parser.add_argument('number_of_credit_problems', type=str)
        parser.add_argument('current_credit_balance', type=str)
        parser.add_argument('maximum_open_credit', type=str)
        parser.add_argument('bankruptcies', type=str)
        parser.add_argument('tax_liens', type=str)
        args = parser.parse_args()
        test_new = pd.read_csv('dataset/train_new.csv')
        test_new["Loan ID"]=args['loan_id']
        test_new["Customer ID"]=args['customer_id']
        test_new["Current Loan Amount"]=int(args['current_loan_amount'])
        test_new["Term"]=args['term']
        test_new["Credit Score"]=int(args['credit_score'])
        test_new["Annual Income"]=int(args['annual_income'])
        test_new["Years in current job"]=args['years_in_current_job']
        test_new["Home Ownership"]=args['home_ownership']
        test_new["Purpose"]=args['purpose']
        test_new["Monthly Debt"]=int(args['monthly_dept'])
        test_new["Years of Credit History"]=float(args['years_of_credit_history'])
        test_new["Months since last delinquent"]=float(args['months_since_last_delinquent'])
        test_new["Number of Open Accounts"]=int(args['number_of_open_accounts'])
        test_new["Number of Credit Problems"]=int(args['number_of_credit_problems'])
        test_new["Current Credit Balance"]=int(args['current_credit_balance'])
        test_new["Maximum Open Credit"]=int(args['maximum_open_credit'])
        test_new["Bankruptcies"]=int(args['bankruptcies'])
        test_new["Tax Liens"]=int(args['tax_liens'])
        test_new_one = callDataone(test_new)
        test_puri_one = sc.fit_transform(test_new_one)
        y_pred = kj_model.predict(test_puri_one[0:1])
        if y_pred[0]==0:
            return "Charge Off"
        elif y_pred[0]==1:
            return "Fully Paid"

api.add_resource(RegistUser, '/add')
 
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2019-12-18 17:28:14,728] ERROR in app: Exception on /add [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\juno\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\envs\juno\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\envs\juno\lib\site-packages\flask_restful\__init__.py", line 458, in wrapper
    resp = resource(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\juno\lib\site-packages\flask\views.py", line 89, in view
    return self.dispatch_request(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\juno\lib\site-packages\flask_restful\__init__.py", line 573, in dispatch_request
    resp = meth(*args, **kwargs)
  File "<ipython-input-9-5200e62a51e4>", line 53, in post
    y_pred = kj_model.p